In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

df = pd.read_csv('dataset_mndsh.csv')
print(df.head())

   ID_TAREFA PRIORIDADE  DIFICULDADE  HORAS_PROD_MED  NIVEL_FOCO_MED  \
0          1      média          2.3             6.6             6.3   
1          2      média          1.2             8.8             8.4   
2          3      baixa          7.5             5.0             8.5   
3          4      média          1.0             6.7             8.6   
4          5      baixa          5.9             6.9             9.4   

   ESTRESSE_MED  HORAS_DORMIDAS_MED  SATISFACAO_MED  CARGA_TRABALHO_MED  \
0           1.4                 6.5             6.5                 9.0   
1           4.3                 7.7             6.7                 4.7   
2           7.0                 6.6             4.8                 6.4   
3           2.8                 5.2             6.1                 7.0   
4           6.8                 6.6             7.9                 5.4   

   TEMPO_CONCLUSAO_DIAS  ATRASO  
0                  2.50       0  
1                  1.53       0  
2             

Separação de Features (X) e Target (Y)

In [ ]:

Y = df['TEMPO_CONCLUSAO_DIAS']

X = df[['PRIORIDADE', 'DIFICULDADE', 'HORAS_PROD_MED', 'NIVEL_FOCO_MED', 
        'ESTRESSE_MED', 'HORAS_DORMIDAS_MED', 'SATISFACAO_MED', 'CARGA_TRABALHO_MED']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:

colunas_numericas = ['DIFICULDADE', 'HORAS_PROD_MED', 'NIVEL_FOCO_MED', 
                     'ESTRESSE_MED', 'HORAS_DORMIDAS_MED', 'SATISFACAO_MED', 'CARGA_TRABALHO_MED']
colunas_categoricas = ['PRIORIDADE']

transformador_numerico = StandardScaler() 

transformador_categorico = OneHotEncoder(handle_unknown='ignore') 

pre_processador = ColumnTransformer(
    transformers=[
        ('num', transformador_numerico, colunas_numericas),
        ('cat', transformador_categorico, colunas_categoricas)
    ],
    remainder='passthrough' 
)

In [ ]:

modelo_regressao = RandomForestRegressor(n_estimators=100, random_state=42)

pipeline_regressao = Pipeline(steps=[('preprocessor', pre_processador),
                                     ('regressor', modelo_regressao)])

pipeline_regressao.fit(X_train, Y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:

Y_pred = pipeline_regressao.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)

r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} dias")
print(f"R-squared Score (R²): {r2:.2f}")

resultados = pd.DataFrame({'Real': Y_test, 'Previsto': Y_pred})
print("\nComparação de valores (Amostra):")
print(resultados.head(10).round(2))

Mean Squared Error (MSE): 1.91
Root Mean Squared Error (RMSE): 1.38 dias
R-squared Score (R²): 0.55

Comparação de valores (Amostra):
      Real  Previsto
1801  0.50      1.12
1190  1.23      2.06
1817  6.19      5.80
251   2.56      2.14
2505  5.21      4.60
1117  1.20      0.96
1411  7.51      6.00
2113  3.59      4.25
408   2.39      2.46
2579  1.35      1.07


In [ ]:

joblib.dump(pipeline_regressao, 'regressao_tempo.joblib')
print("Modelo de Regressão salvo como 'regressao_tempo.joblib'")

Modelo de Regressão salvo como 'regressao_tempo.joblib'
